<a href="https://colab.research.google.com/github/hyxxnii/Tave-6th-Project/blob/master/%EC%A4%91%EA%B3%A0%EC%B0%A8%20%ED%8C%90%EB%A7%A4%EA%B8%B0%EA%B0%84%20%EC%98%88%EC%B8%A1%20%EB%AA%A8%EB%8D%B8%20%5BKB%EC%BA%90%ED%94%BC%ED%83%88%5D%20%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

train_path = '../data/.train/.task147/train.csv'
test_path = '../data/.train/.task147/test.csv'
submission_path = '../data/.sample_submission/.task147/sample_submission.csv'
car_df_train = pd.read_csv(train_path)
car_df_submission= pd.read_csv(submission_path)
car_df = car_df_train.copy()
car_df.head()

In [ ]:
# SalePrice 로그 변환
originial_Periods = car_df['ad_periods']
car_df['ad_periods'] = np.log1p(car_df['ad_periods'])

In [ ]:
car_va = car_df.filter(regex='^va', axis=1)
cols = car_va.columns

for col in cols:
    fig,ax = plt.subplots(1,1)
    sns.distplot(car_df[col])

In [ ]:
col = ['va04', 'va06', 'va07', 'va13', 'va25', 'va28','va12','va33']

for c in col:
    fig,ax = plt.subplots(1,1)
    sns.distplot(car_df[c])

In [ ]:
cond1 = car_df['va07'] > 14
cond2 = car_df['ad_periods'] > 1
outlier_index = car_df[cond1 & cond2].index
print('va07 이상치 레코드 index: ', outlier_index.values)
# 이상치를 해당 변수의 중간값으로 대체
car_df['va07'][outlier_index] = np.median(car_df['va07'])

cond1 = car_df['va22'] >14.0
cond2 = car_df['ad_periods'] > 3
outlier_index = car_df[cond1 & cond2].index
print('va22 이상치 레코드 index: ', outlier_index.values)
# 이상치를 해당 변수의 중간값으로 대체
car_df['va22'][outlier_index] = np.median(car_df['va22'])

cond1 = car_df['va23'] > 15
cond2 = car_df['ad_periods'] > 3
outlier_index = car_df[cond1 & cond2].index
print('va23 이상치 레코드 index: ', outlier_index.values)
# 이상치를 해당 변수의 중간값으로 대체
car_df['va23'][outlier_index] = np.median(car_df['va23'])

cond1 = car_df['va25'] > 2.5
cond2 = car_df['va25'] < 5.0
outlier_index = car_df[cond1 & cond2].index
print('va25 이상치 레코드 index: ', outlier_index.values)
# 이상치를 해당 변수의 중간값으로 대체
car_df['va25'][outlier_index] = np.median(car_df['va25'])


In [ ]:
car_df.reset_index(drop=True, inplace=True)
car_df['ad_periods']

In [ ]:
# 원핫 인코딩 va05, va24, va32
cond1 = car_df['va32'] == 5
cond2 = car_df['va24'] == 5

outlier_index = car_df[cond1 | cond2].index

print('이상치 레코드 index: ', outlier_index.values)
print('이상치 삭제 전 car_df shape: ', car_df.shape)

# DataFrame의 인덱스를 이용해 이상치 레코드 삭제
car_df = car_df.drop(outlier_index, axis=0).reset_index(drop=True)
print('이상치 삭제 후 car_df shape: ', car_df.shape)

# category형으로 추정되는 변수 원핫 인코딩
# va05, va24, va32
car_df_ohe = pd.get_dummies(car_df, columns=['va05','va24','va32'])
car_df_ohe.head()

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(model):
    pred = np.expm1(model.predict(X_test))
    mse = mean_squared_error(y_test_exp, pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, 'RMSE: ', np.round(rmse,3))
    return rmse

def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return np.round(rmses, 3)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor

y_target = car_df_ohe['ad_periods'] # 로그 변환됨
X_features = car_df_ohe.drop('ad_periods', axis=1, inplace=False) # inplace=False 주의
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156)

y_test_exp = np.expm1(y_test)

xgb_reg2 = XGBRegressor(n_estimators=1000, learning_rate=0.05,
                      colsample_bytree=0.5, subsample=0.8, random_state=156)

xgb_reg2.fit(X_train, y_train, early_stopping_rounds=100,
           eval_metric='rmse', eval_set=[(X_train, y_train), (X_test, y_test)])

get_rmse(xgb_reg2)

In [ ]:
car_df_test = pd.read_csv(test_path)
car_df_test.head()

In [ ]:
col = ['va04', 'va06', 'va07', 'va13', 'va25', 'va28','va12','va33']
car_df_test[col] = np.log1p(car_df_test[col])

In [ ]:
# 원핫 인코딩 va05, va24, va32
car_df_test_ohe = pd.get_dummies(car_df_test, columns=['va05','va24','va32'])
car_df_test_ohe.head()

In [ ]:
# 피처 데이터 생성후 model predict
test = car_df_test_ohe[list(car_df_test_ohe.columns)]
result = np.expm1(xgb_reg2.predict(test))

In [ ]:
# submit 파일 작성
submission = pd.read_csv(submission_path)
submission['ad_periods'] = [round(x) for x in result]
submission.head()

In [ ]:
submission.to_csv('./submit_yh4.csv', index=False)

In [ ]:
from nipa.taskSubmit import nipa_submit

team_id="1345"
task_no="147"
nipa_submit(team_id=team_id,
            task_no=task_no,
            result='./submit_yh4.csv')